In [1]:
!pip install rake_nltk
!pip install pymorphy3
!pip install pymorphy3-dicts-ru

# !pip uninstall numpy
# !pip install numpy==1.26.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 74.3 MB/s eta 0:00:00


In [ ]:
# !pip show numpy

In [ ]:
# !pip install surprise

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import ast
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import rake_nltk
from rake_nltk import Rake
import nltk

import re
from nltk.corpus import stopwords
import pymorphy3
# from surprise import Reader, Dataset, SVD, dump
# from surprise.model_selection import cross_validate
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
import pickle
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, T5EncoderModel
from sentence_transformers import SentenceTransformer

# from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# if there is need to preprocess data from the beginning
movie_df = pd.read_csv('/content/drive/MyDrive/Kinopoisk/movie_df_small.csv')


In [28]:
# upload saved preprocessed data
data = pd.read_csv('/content/drive/MyDrive/Kinopoisk/data.csv')

In [29]:
data

,Unnamed: 0,id,name,genres,persons,description,soup
0,0,298,Люди Икс 2,фантастика боевик триллер,брайансингер патрикстюарт хьюджекман иэнмаккел...,мутант продолжать борьба против общество котор...,брайансингер патрикстюарт хьюджекман иэнмаккел...
1,1,299,Матрица: Перезагрузка,фантастика боевик,лилливачовски киануривз лоренсфишбёрн хьюгоуив...,борец свобода нео тринити морфеус продолжать р...,лилливачовски киануривз лоренсфишбёрн хьюгоуив...
2,2,300,Лиззи Магуайр,мелодрама комедия приключения семейный музыка,джимфолл хиларидафф адамлэмберг халлиморган ро...,тринадцатилетний школьница лиззь магуайер прия...,джимфолл хиларидафф адамлэмберг халлиморган ро...
3,3,301,Матрица,фантастика боевик,лилливачовски киануривз лоренсфишбёрн кэрри-эн...,жизнь томас андерсон разделить два часть день ...,лилливачовски киануривз лоренсфишбёрн кэрри-эн...
4,4,302,Клад,драма комедия детектив приключения семейный,эндрюдэвис сигурниуивер джонвойт патришааркетт...,на семья подросток имя стенли илнетс прочесть ...,эндрюдэвис сигурниуивер джонвойт патришааркетт...
...,...,...,...,...,...,...,...
15793,15793,54859,Все дело в воде,мелодрама комедия,келлихерд кериджочэпман терезагаррет дерриксан...,действие фильм происходить маленький консерват...,келлихерд кериджочэпман терезагаррет дерриксан...
15794,15794,54866,Слизни,ужасы,хуанпикесимон майклгарфилд кимтерри филипмакхе...,в небольшой городок начать исчезать житель про...,хуанпикесимон майклгарфилд кимтерри филипмакхе...
15795,15795,54874,Замужество Марии Браун,драма,райнервернерфассбиндер ханнашигулла клауслёвич...,проводить муж восточный фронт мария начинать с...,райнервернерфассбиндер ханнашигулла клауслёвич...
15796,15796,54875,В год тринадцати лун,драма,райнервернерфассбиндер фолькершпенглер ингридк...,« каждый год это год луна » говориться туманно...,райнервернерфассбиндер фолькершпенглер ингридк...


In [36]:
def get_recommendations(idx):
    idx = indices[idx]
    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    # print(sim_scores)
    movie_indices = [i[0] for i in sim_scores]
    return names.iloc[movie_indices]
    # return movie_indices

In [6]:
def main_cast(persons):
  persons = ast.literal_eval(persons)
  main_cast = []
  director_name = ''
  for item in persons:
    if item['name'] != None:
      main_cast.append(item['name'].replace(' ', ''))
    if item['enProfession'] == 'director':
      director_name = item['name']
  if len(main_cast) >=6:
    main_cast = main_cast[:6:]
  else:
    main_cast = main_cast[:len(main_cast):]
  if director_name != None:
    main_cast.insert(0, director_name.replace(' ', ''))
  else:
    main_cast.insert(0, 'unknown')
  return ' '.join(main_cast)

In [7]:
def genres_cleanup(genres):
  genres = ast.literal_eval(genres)
  genres_list = []
  for item in genres:
    genres_list.append(item['name'])
    # print(item)
  return ' '.join(genres_list)

In [8]:
def description_cleanup(orig_tokens):
  try:
    morph = pymorphy3.MorphAnalyzer()
    patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
    key_words = []
    tokens = nltk.tokenize.word_tokenize(orig_tokens)
    # print(tokens)
    for token in tokens:
      token = re.sub(patterns, '', token)
      if token not in stopwords.words("russian"):
        if morph.parse(token)[0].normal_form != '':
          key_words.append(morph.parse(token)[0].normal_form)
          # print(morph.parse(token)[0].normal_form)
      # print(stopwords.words("russian"))
  except:
    print(orig_tokens)
    return 'unknown'
  return ' '.join(key_words)

In [ ]:
data = movie_df[['id','name', 'genres', 'persons', 'description']]
data.loc[:,'persons'] = data['persons'].apply(main_cast)
data.loc[:, 'persons'] = data['persons'].str.lower()
data.loc[:, 'genres'] = data['genres'].apply(genres_cleanup)
data.loc[:, 'description'] = data['description'].apply(description_cleanup)
# data.drop(columns='persons', inplace=True)

In [9]:
data['soup'] = data['persons']+' '+ data['genres'] + ' ' + data['description']

In [ ]:
data.to_csv('/content/drive/MyDrive/Kinopoisk/data.csv')

In [10]:
model = SentenceTransformer("ai-forever/FRIDA")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/509 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/54.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.70M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.59M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [11]:
embeddings = model.encode(data['soup'], convert_to_tensor=False)

In [12]:
cosine_sim = cosine_similarity(embeddings, embeddings)

In [ ]:

with open('/content/drive/MyDrive/Kinopoisk/cosine_sim.npy', 'wb') as f:
  np.save(f, cosine_sim)

In [24]:
data = data.reset_index()

ids = data['id']
with open('/content/drive/MyDrive/Kinopoisk/ids.pkl', 'wb') as f:
  pickle.dump(ids, f)


indices = pd.Series(data.index, index=data['id'])
with open('/content/drive/MyDrive/Kinopoisk/indices.pkl', 'wb') as f:
  pickle.dump(indices, f)


In [30]:
data = data.reset_index()
names = data['name']
indices = pd.Series(data.index, index=data['name'])

In [33]:
indices

,0
name,
Люди Икс 2,0
Матрица: Перезагрузка,1
Лиззи Магуайр,2
Матрица,3
Клад,4
...,...
Все дело в воде,15793
Слизни,15794
Замужество Марии Браун,15795


In [45]:
movie_indice = 3

print(f'Content based recommendation to movie {names.iloc[movie_indice]}')
get_recommendations(movie_indice)


Content based recommendation to movie Матрица


<ipython-input-36-2006617800>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[idx]


,name
17,Матрица: Революция
1,Матрица: Перезагрузка
2243,Цепная реакция
2381,Адвокат дьявола
5990,Крепкий орешек 4.0


In [ ]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2), min_df=0.0)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfid = TfidfVectorizer(
    min_df = 3,
    max_features = None,
    analyzer = "word",
    ngram_range = (1,3)
)

tfid_matrix = tfid.fit_transform(data['soup'])

tfid_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 919031 stored elements and shape (15798, 58234)>

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel

sig = sigmoid_kernel(tfid_matrix, tfid_matrix)

sig[0]

array([0.76159887, 0.76159433, 0.76159417, ..., 0.76159416, 0.76159416,
       0.76159416])

In [ ]:
count_matrix = count.fit_transform(data['soup'])

In [ ]:
count_matrix

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 1000875 stored elements and shape (10000, 453048)>

In [ ]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
cosine_sim = cosine_similarity(tfid_matrix, tfid_matrix)